## Setup Models

In [ ]:
!pip install langchain_openai
!pip install langchain_chroma
!pip install langchain_community
!pip install langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 62.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 83.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 14.5 MB/s eta 0:

In [ ]:
from langchain_openai import AzureChatOpenAI
from langchain_openai import AzureOpenAIEmbeddings
import os

#Setup the environment variables
os.environ["AZURE_OPENAI_API_KEY"]="<YOUR KEY>"
os.environ["AZURE_OPENAI_ENDPOINT"]="<YOUR END POINT>"

#Setup the LLM
model = AzureChatOpenAI(
    azure_deployment="gpt-4o" ,
    api_version="2024-12-01-preview",
    model="gpt-4o"
)

'''The text-embedding-3-large model, released by OpenAI, is a high-performance text embedding model designed to convert text into a numerical representation (embeddings) that
captures semantic meaning and relationships between words and phrases. This model is particularly useful for tasks where accuracy and detail are paramount, offering improved
performance on benchmarks like MIRACL and MTEB compared to its predecessors. '''

#Setup the Embedding
embedding = AzureOpenAIEmbeddings(
    model="text-embedding-3-large",
    openai_api_version="2024-02-01"
)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 03.02. Add  Product Pricing function tool

In [ ]:
import pandas as pd
from langchain_core.tools import tool

#Load the laptop product pricing CSV into a Pandas dataframe.
product_pricing_df = pd.read_csv("/content/drive/MyDrive/LinkedIn Learning/LangChain/Data/Laptop pricing.csv")
print(product_pricing_df)


            Name  Price  ShippingDays
0  AlphaBook Pro   1499             2
1     GammaAir X   1399             7
2  SpectraBook S   2499             7
3   OmegaPro G17   2199            14
4  NanoEdge Flex   1699             2


In [ ]:
@tool
def get_laptop_price(laptop_name:str) -> int :
    #docstring
    """
    This function returns the price of a laptop, given its name as input.
    It performs a substring match between the input name and the laptop name.
    If a match is found, it returns the price of the laptop.
    If there is NO match found, it returns -1
    """

    #Filter Dataframe for matching names
    match_records_df = product_pricing_df[
                        product_pricing_df["Name"].str.contains(
                                                "^" + laptop_name, case=False)
                        ]
    #Check if a record was found, if not return -1
    if len(match_records_df) == 0 :
        return -1
    else:
        return match_records_df["Price"].iloc[0]

#Test the tool. Before running the test, comment the @tool annotation
#print(get_laptop_price("alpha"))
#print(get_laptop_price("testing"))

## 03.03. Add Product Features Retrieval Tool

In [ ]:
!pip install pysqlite3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.7/40.7 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pysqlite3: filename=pysqlite3-0.5.4-cp311-cp311-linux_x86_64.whl size=159238 sha256=76732d6a2274e0962fc10b318f2a76f60d1caf804ea7ea19063c3870497347af
  Stored in directory: /root/.cache/pip/wheels/83/05/4e/8fb9d7378ff72e4fd02b69df7a6052c34d33cea2a0231ca232
Successfully built pysqlite3


In [ ]:
!pip install pypdf #PyPDFLoader is designed to use the pypdf library under the hood

In [ ]:
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3') #mapping pysqllite3 to sqllite3 for the chrome DB to work

from langchain.tools.retriever import create_retriever_tool
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
# Load, chunk and index the contents of the product featuers document.
loader=PyPDFLoader("/content/drive/MyDrive/LinkedIn Learning/LangChain/Data/Laptop product descriptions.pdf")
docs = loader.load()

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=256)
splits = text_splitter.split_documents(docs)

''' A vector database is a specialized type of database designed to store, manage, and query high-dimensional vector data, which are
numerical representations of unstructured data like text, images, or audio. These databases are optimized for similarity searches,
allowing for efficient retrieval of data points based on how closely they match a given query vector. '''

#Create a vector store (DB) with Chroma in-memory db
prod_feature_store = Chroma.from_documents(
    documents=splits,
    embedding=embedding
)

get_product_features = create_retriever_tool(
    prod_feature_store.as_retriever(search_kwargs={"k": 1}),
    name="Get_Product_Features",
    description="""
    This store contains details about Laptops. It lists the available laptops
    and their features including CPU, memory, storage, design and advantages
    """
)

#Test the product feature store
#print(prod_feature_store.as_retriever().invoke("Tell me about the AlphaBook Pro") )

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


## 03.04.Setup a Product QnA chatbot

In [ ]:
from langgraph.prebuilt import create_react_agent #To create ChatBot and Agent
from langgraph.checkpoint.memory import MemorySaver #provide conversational memory
from langchain_core.messages import AIMessage,HumanMessage,SystemMessage

#Create a list of tools available
tools = [get_laptop_price, get_product_features]

#Create memory across questions in a conversation (conversation memory)
checkpointer=MemorySaver()

#Create a Product QnA Agent. This is actual a graph in langGraph
product_QnA_agent=create_react_agent(
                                model=model, #LLM to use
                                tools=tools, #List of tools to use
                                debug=False, #Debugging turned on if needed
                                checkpointer=checkpointer, #For conversation memory
)

In [ ]:
#Create a System prompt to provide a persona to the chatbot
system_prompt = SystemMessage("""
    You are professional chatbot that answers questions about laptops sold by your company.
    To answer questions about laptops, you will ONLY use the available tools and NOT your own memory.
    You will handle small talk and greetings by producing professional responses.
    """
)

In [ ]:
#Setup chatbot
import uuid
#To maintain memory, each request should be in the context of a thread.
#Each user conversation will use a separate thread ID
config = {"configurable": {"thread_id": uuid.uuid4()}}

#Test the agent with an input
inputs = {"messages":[
                system_prompt,
                HumanMessage("What are the features and pricing for GammaAir?")
            ]}

#Use streaming to print responses as the agent  does the work.
#This is an alternate way to stream agent responses without waiting for the agent to finish
for stream in product_QnA_agent.stream(inputs, config, stream_mode="values"):
    message=stream["messages"][-1]
    if isinstance(message, tuple):
        print(message)
    else:
        message.pretty_print()

================================ Human Message =================================

What are the features and pricing for GammaAir?
================================== Ai Message ==================================
Tool Calls:
  Get_Product_Features (call_7R1iB6oY8MPIxcmIQsyFn30C)
 Call ID: call_7R1iB6oY8MPIxcmIQsyFn30C
  Args:
    query: GammaAir
  get_laptop_price (call_Se7fI1oQeHs3EYQYq3eNLlVR)
 Call ID: call_Se7fI1oQeHs3EYQYq3eNLlVR
  Args:
    laptop_name: GammaAir


ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


================================= Tool Message =================================
Name: get_laptop_price

1399
================================== Ai Message ==================================

The GammaAir X features the following specifications:

- **Processor:** AMD Ryzen 7
- **Memory:** 32GB of DDR4 RAM
- **Storage:** 512GB NVMe SSD
- **Design:** Thin and light form factor, ideal for portability.

It is priced at **$1399**.


## 03.05. Execute the Product QnA Chatbot

In [ ]:
import uuid
#Send a sequence of messages to chatbot and get its response
#This simulates the conversation between the user and the Agentic chatbot
user_inputs = [
    "Hello",
    "I am looking to buy a laptop",
    "Give me a list of available laptop names",
    "Tell me about the features of  SpectraBook",
    "How much does it cost?",
    "Give me similar information about OmegaPro",
    "What info do you have on AcmeRight ?",
    "Thanks for the help"
]

#Create a new thread - helps maintain the conversational memory of the chatbot for each question
config = {"configurable": {"thread_id": str(uuid.uuid4())}}

for input in user_inputs:
    print(f"----------------------------------------\nUSER : {input}")
    #Format the user message
    user_message = {"messages":[HumanMessage(input)]}
    #Get response from the agent
    ai_response = product_QnA_agent.invoke(user_message,config=config)
    #Print the response
    print(f"AGENT : {ai_response['messages'][-1].content}")


----------------------------------------
USER : Hello
AGENT : Hi there! How can I assist you today?
----------------------------------------
USER : I am looking to buy a laptop
AGENT : Great! I can help you with that. 

Could you tell me more about your needs? For example:
- What's your budget?
- What features are important to you (like CPU, RAM, storage, portability)?
- Do you have specific brands or models in mind? 

Let me know, and I'll help you find the perfect laptop!
----------------------------------------
USER : Give me a list of available laptop names
AGENT : Here are the available laptops:

1. **AlphaBook Pro** - A sleek ultrabook with a 12th Gen Intel i7 processor, 16GB RAM, and 1TB SSD.
2. **GammaAir X** - Features AMD Ryzen 7, 32GB RAM, and a 512GB NVMe SSD, designed for portability and high performance.
3. **SpectraBook S** - Includes Intel Core i9, 64GB RAM, and 2TB SSD, ideal for power users with intensive tasks.
4. **OmegaPro G17** - A gaming laptop with Ryzen 9 5900H

In [ ]:
#conversation memory by user
def execute_prompt(user, config, prompt):
    inputs = {"messages":[("user",prompt)]}
    ai_response = product_QnA_agent.invoke(inputs,config=config)
    print(f"\n{user}: {ai_response['messages'][-1].content}")

#Create different session threads for 2 users
config_1 = {"configurable": {"thread_id": str(uuid.uuid4())}}
config_2 = {"configurable": {"thread_id": str(uuid.uuid4())}}

#Test both threads
execute_prompt("USER 1", config_1, "Tell me about the features of  SpectraBook")
execute_prompt("USER 2", config_2, "Tell me about the features of  GammaAir")
execute_prompt("USER 1", config_1, "What is its price ?")
execute_prompt("USER 2", config_2, "What is its price ?")




USER 1: The SpectraBook S is designed for power users and delivers exceptional performance for intensive tasks such as video editing and 3D rendering. Its key features include:

- **Processor:** Intel Core i9 (high-performance CPU for demanding tasks)
- **Memory:** 64GB RAM (ample capacity for multitasking and running complex operations)
- **Storage:** Massive 2TB SSD (provides high-speed storage for large files)
- **Design:** Workstation-class laptop

This laptop is perfect for individuals who need a reliable machine for heavy workloads and professional-grade applications.

USER 2: The GammaAir X is a high-performance laptop that consists of the following features:

- **Processor**: AMD Ryzen 7
- **Memory**: 32GB of DDR4
- **Storage**: 512GB NVMe SSD
- **Design**: Thin and lightweight form factor

This laptop is perfect for users who prioritize portability along with exceptional computing power.

USER 1: The price of the SpectraBook S is $2,499.

USER 2: The price of the GammaAir X l